Lavoro sui dataset iniziali con tutte le etichette

In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import sys
sys.path.insert(1, '')
from utility import Dataset

train_df = pd.read_csv(f'KDDTrain+.txt', header=None)
test_df = pd.read_csv(f'KDDTest+.txt', header=None)

columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
           'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted',
           'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login',
           'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
           'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count',
           'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
           'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
           'dst_host_srv_rerror_rate', 'label', 'score']
train_df.columns = columns
test_df.columns = columns

nominal_features = ['protocol_type', 'service', 'flag']
binary_features = ['land', 'logged_in', 'root_shell', 'su_attempted', 'is_host_login', 'is_guest_login']
numeric_features = [feature for feature in columns if feature not in nominal_features + binary_features + ['label', 'score']]

# 'su_attempted' dovrebbe essere binario ma ha 3 valori. Sostituiamo '2.0' con '0.0'
train_df['su_attempted'] = train_df['su_attempted'].replace(2, 0)
test_df['su_attempted'] = test_df['su_attempted'].replace(2, 0)

# 'num_outbound_cmds' ha sempre lo stesso valore quindi possiamo dropparla
train_df = train_df.drop('num_outbound_cmds', axis=1)
test_df = test_df.drop('num_outbound_cmds', axis=1)

** remove highly uncorrelated features ?? **

# One Hot Encoding for categorical variables
trovare una motivazione al perchè ho scelto questa codifica

In [53]:
enc = OneHotEncoder()

train_encoded_array = enc.fit_transform(train_df[nominal_features]).toarray()
test_encoded_array = enc.transform(test_df[nominal_features]).toarray()

new_columns = []
for i, feature in enumerate(nominal_features):
    new_columns.extend([f"{feature}_{str(cat)}" for cat in enc.categories_[i]])

train_ohe = train_df.drop(nominal_features, axis=1)
train_ohe = pd.concat([train_ohe, pd.DataFrame(train_encoded_array, columns=new_columns)], axis=1)

test_ohe = test_df.drop(nominal_features, axis=1)
test_ohe = pd.concat([test_ohe, pd.DataFrame(test_encoded_array, columns=new_columns)], axis=1)

print(train_ohe)
print(test_ohe)


        duration  src_bytes  dst_bytes  land  wrong_fragment  urgent  hot  \
0              0        491          0     0               0       0    0   
1              0        146          0     0               0       0    0   
2              0          0          0     0               0       0    0   
3              0        232       8153     0               0       0    0   
4              0        199        420     0               0       0    0   
...          ...        ...        ...   ...             ...     ...  ...   
125968         0          0          0     0               0       0    0   
125969         8        105        145     0               0       0    0   
125970         0       2231        384     0               0       0    0   
125971         0          0          0     0               0       0    0   
125972         0        151          0     0               0       0    0   

        num_failed_logins  logged_in  num_compromised  root_shell  \
0     

# Feature Selection 

1. Attribute Ratio

2. Correlation-based Feature Selection

3. Information Gain

4. Gain Ratio

5. PCA

# 1. Attribute Ratio
Attribute Ratio approach is used for feature selection purposes. This approach was described by Hee-su Chae and Sang Hyun Choi in Feature Selection for efficient Intrusion Detection using Attribute Ratio and Feature Selection for Intrusion Detection using NSL-KDD

This approach is also used for nominal variables as they were encoded as binary variables above.

As it is a possible to have 'null' values because binary features could have Frequency(0) = 0, those 'null' values are replaced with 1000.0 (magic number). For NSL KDD dataset it is related only for 'protocol_type_tcp' ohe variable.

In section 4, we explain NSL-KDD data which has three attribute types. We use attribute average and frequency for each class calculate the AR from numeric and binary type. AR can be calculated as :                             (7)   Class Ratio (CR) is attribute is ratio of each class for Attribute i. CR is calculated by two methods according to the type of attributes. CR can be calculated as for numeric :                                      (8)  CR can be calculated as for binary : 

After calculating AR(i), Features rank ordering larger AR. Table 4 shows the rank of features with a calculated AR. 

In [66]:
# ho calcolato l'attribute ratio per ogni feature (divisione in 5 classi)
train_5_df = Dataset(train_df).get_label5()

def attributeRatio(feature, dataset):
    class_ratio = {}

    if feature in nominal_features:
        return 0

    if feature in binary_features:
        for label in dataset['label'].unique():
            class_ones = dataset[dataset['label'] == label][feature].sum()
            class_zeros = dataset[dataset['label'] == label][feature].count() - class_ones
            class_ratio[label] = class_ones / class_zeros
    else:
        total_mean = dataset[feature].mean()
        for label in dataset['label'].unique():
            class_mean = dataset[dataset['label'] == label][feature].mean()
            class_ratio[label] = class_mean / total_mean
    return max(class_ratio.values())

# calcola l'attribute ratio per ogni feature tranne 'label' e 'score'
attribute_ratios = {}
for feature in train_5_df.columns:
    if feature not in ['label', 'score', 'num_outbound_cmds']:
        attribute_ratios[feature] = attributeRatio(feature, train_5_df)

# ordina le feature in base all'attribute ratio
sorted_attribute_ratios = sorted(attribute_ratios.items(), key=lambda x: x[1], reverse=True)
# stampa una tabella con le feature ordinate. la prima colonna è il rank, la seconda la feature e la terza l'attribute ratio
for i, (feature, ratio) in enumerate(sorted_attribute_ratios):
    print(f"{i+1} & {feature} & {ratio:.2f} \\\\")


1 & num_shells & 326.11 \\
2 & urgent & 173.04 \\
3 & num_file_creations & 62.23 \\
4 & num_failed_logins & 46.04 \\
5 & hot & 40.77 \\
6 & logged_in & 10.57 \\
7 & dst_bytes & 9.15 \\
8 & src_bytes & 8.46 \\
9 & duration & 7.23 \\
10 & dst_host_srv_diff_host_rate & 5.76 \\
11 & dst_host_diff_srv_rate & 4.84 \\
12 & num_access_files & 4.69 \\
13 & dst_host_same_src_port_rate & 4.39 \\
14 & num_compromised & 4.34 \\
15 & diff_srv_rate & 4.07 \\
16 & dst_host_srv_rerror_rate & 3.67 \\
17 & srv_rerror_rate & 3.67 \\
18 & rerror_rate & 3.65 \\
19 & dst_host_rerror_rate & 3.28 \\
20 & srv_diff_host_rate & 3.08 \\
21 & wrong_fragment & 2.74 \\
22 & dst_host_srv_serror_rate & 2.67 \\
23 & srv_serror_rate & 2.64 \\
24 & serror_rate & 2.63 \\
25 & dst_host_serror_rate & 2.63 \\
26 & num_root & 2.61 \\
27 & count & 2.12 \\
28 & dst_host_srv_count & 1.65 \\
29 & dst_host_same_srv_rate & 1.56 \\
30 & same_srv_rate & 1.51 \\
31 & dst_host_count & 1.34 \\
32 & srv_count & 1.18 \\
33 & root_shell & 1